# Task 2

In [1]:
# import dependencies

import pandas as pd
import matplotlib.pyplot as plt

from keras import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, AveragePooling2D
from keras.optimizers import Adam, Nadam, RMSprop, SGD, Adagrad, Adadelta, Adamax
from keras.callbacks import Callback, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split

In [2]:
# loading dataset
data = pd.read_csv('../datasets/train.csv')

In [3]:
# extracting labels
X_data = data.drop(['label'], axis=1)
Y_data = data['label']

In [4]:
# splitting data for train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=2)

In [5]:
# normalizing data
X_train = X_train / 255.0
X_val = X_val / 255.0

Y_train = to_categorical(Y_train, num_classes=10)
Y_val = to_categorical(Y_val, num_classes=10)

In [6]:
# reshaping data
X_train = X_train.values.reshape(-1,28,28,1)
X_val = X_val.values.reshape(-1,28,28,1)

# Comparing with LeNet-5

In [7]:
# initializing LeNet-5

lenet_model = Sequential()
lenet_model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu',
                 input_shape=(28, 28, 1)))
lenet_model.add(AveragePooling2D(strides=2))
lenet_model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu')
          )
lenet_model.add(AveragePooling2D(strides=2))
lenet_model.add(Flatten())
lenet_model.add(Dense(256, activation='relu'))
lenet_model.add(Dense(84, activation='relu'))
lenet_model.add(Dense(10, activation='softmax'))

In [8]:
# compile LeNet model
lenet_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=5e-4))

In [9]:
# initializing reduce lr callback
lenet_reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', patience=3, verbose=1, factor=0.2, min_lr=1e-6)

In [10]:
data_generator = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1)
data_generator.fit(X_train)

In [ ]:
# fit LeNet model
lenet_model_history = lenet_model.fit(data_generator.flow(X_train, Y_train, batch_size=100), steps_per_epoch=len(X_train) / 100, epochs=30, validation_data=(X_val, Y_val), callbacks=[lenet_reduce_lr], verbose=0)

2022-11-13 22:19:10.692267: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [ ]:
# printing LeNet model's score
score = lenet_model.evaluate(X_val, Y_val, batch_size=32)

print(score)

In [ ]:
# initializing the best model from previous task

standard_model = Sequential()
standard_model.add(Conv2D(filters=8, kernel_size=(3, 6), padding='same', activation='relu', input_shape=(28, 28, 1)))
standard_model.add(MaxPooling2D(pool_size=(2, 2)))
standard_model.add(Conv2D(filters=16, kernel_size=(10, 5), padding='same',activation='relu')),
standard_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),)
standard_model.add(Conv2D(filters=16, kernel_size=(3, 6), padding='same', activation='relu'))
standard_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
standard_model.add(Flatten())
standard_model.add(Dense(256, activation='relu'))
standard_model.add(Dropout(0.25))
standard_model.add(Dense(10, activation='softmax'))

standard_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam(learning_rate=0.0005))

standard_model_history = standard_model.fit(data_generator.flow(X_train, Y_train, batch_size=256), epochs=50, validation_data=(X_val, Y_val), verbose=0)

In [ ]:
# printing standard model's score
score = standard_model.evaluate(X_val, Y_val, batch_size=32)

print(score)

In [ ]:
plt.figure()
plt.title("LeNet vs Standard")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.plot(lenet_model_history.history['accuracy'])
plt.plot(lenet_model_history.history['val_accuracy'])
plt.plot(standard_model_history.history['accuracy'])
plt.plot(standard_model_history.history['val_accuracy'])
plt.legend(['LeNet Accuracy', 'LeNet Validation Accuracy', 'Standard Accuracy', 'Standard Validation Accuracy'], loc='lower right')
plt.show()